In [1]:
import pandas as pd
import numpy as np
import zipfile
import os
from typing import List, Optional

def dividir_csv_em_n_partes(arquivo_original: str, pasta_saida: str, n_partes: int) -> Optional[List[str]]:
    """
    Divide um arquivo CSV em N partes aproximadamente iguais.

    Args:
        arquivo_original (str): O caminho para o arquivo CSV a ser dividido.
        pasta_saida (str): O diretório onde as partes divididas serão salvas.
        n_partes (int): O número de partes para dividir o arquivo.

    Returns:
        Optional[List[str]]: Uma lista contendo os caminhos para os arquivos CSV criados, ou None em caso de erro.
    """
    try:
        print(f"Lendo o arquivo original: '{arquivo_original}'...")
        df = pd.read_csv(arquivo_original)
        print(f"Dimensões do DataFrame original ('{os.path.basename(arquivo_original)}'): {df.shape[0]} linhas, {df.shape[1]} colunas")
        print("-" * 30)

        # Garante que a pasta de saída exista
        os.makedirs(pasta_saida, exist_ok=True)

        # Usa numpy.array_split para dividir o dataframe em N pedaços de forma eficiente.
        # Esta função lida com casos onde o número de linhas não é perfeitamente divisível por n_partes.
        dfs_divididos = np.array_split(df, n_partes)

        caminhos_arquivos_saida = []
        # Usa o nome do arquivo original como base para os nomes das partes
        nome_base = os.path.splitext(os.path.basename(arquivo_original))[0]

        print(f"Dividindo em {n_partes} partes na pasta '{pasta_saida}'...")
        for i, df_parte in enumerate(dfs_divididos):
            arquivo_saida = os.path.join(pasta_saida, f"{nome_base}_parte_{i+1}_de_{n_partes}.csv")
            print(f"Salvando Parte {i+1}: '{os.path.basename(arquivo_saida)}' ({len(df_parte)} linhas)")
            df_parte.to_csv(arquivo_saida, index=False)
            caminhos_arquivos_saida.append(arquivo_saida)

        print(f"\nArquivo '{arquivo_original}' dividido com sucesso!")
        return caminhos_arquivos_saida

    except FileNotFoundError:
        print(f"Erro: O arquivo '{arquivo_original}' não foi encontrado.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro ao dividir o arquivo: {e}")
        return None

def compactar_lista_csv(lista_arquivos_csv: List[str]) -> List[str]:
    """
    Compacta uma lista de arquivos CSV em arquivos ZIP individuais.

    Args:
        lista_arquivos_csv (List[str]): A lista de caminhos dos arquivos CSV para compactar.

    Returns:
        List[str]: Uma lista dos caminhos dos arquivos ZIP criados.
    """
    caminhos_zip_criados = []
    print("\n--- Etapa: Compactando os CSVs divididos ---")
    for arquivo_csv in lista_arquivos_csv:
        try:
            if not os.path.exists(arquivo_csv):
                print(f"Aviso: Arquivo de entrada '{arquivo_csv}' não encontrado. O ZIP não será criado.")
                continue

            # Define o nome do arquivo ZIP de saída na mesma pasta do CSV
            nome_base = os.path.splitext(arquivo_csv)[0]
            arquivo_zip_saida = f"{nome_base}.zip"

            with zipfile.ZipFile(arquivo_zip_saida, 'w', zipfile.ZIP_DEFLATED) as zipf:
                zipf.write(arquivo_csv, os.path.basename(arquivo_csv))
            print(f"Arquivo '{os.path.basename(arquivo_csv)}' compactado em '{os.path.basename(arquivo_zip_saida)}'")
            caminhos_zip_criados.append(arquivo_zip_saida)
        except Exception as e:
            print(f"Ocorreu um erro ao compactar o arquivo '{arquivo_csv}': {e}")
    return caminhos_zip_criados

def juntar_n_csvs_em_dataframe(lista_arquivos_csv: List[str]) -> Optional[pd.DataFrame]:
    """
    Junta múltiplos arquivos CSV em um único DataFrame do pandas.

    Args:
        lista_arquivos_csv (List[str]): Uma lista de caminhos para os arquivos CSV a serem juntados.

    Returns:
        Optional[pd.DataFrame]: O DataFrame concatenado ou None se ocorrer um erro.
    """
    try:
        dataframes = []
        print("\n--- Etapa: Juntando os CSVs em um DataFrame ---")
        for arquivo_csv in lista_arquivos_csv:
            print(f"Lendo o arquivo '{os.path.basename(arquivo_csv)}'...")
            df_parte = pd.read_csv(arquivo_csv)
            print(f"  > Dimensões: {df_parte.shape[0]} linhas, {df_parte.shape[1]} colunas")
            dataframes.append(df_parte)

        if not dataframes:
            print("Nenhum DataFrame para concatenar.")
            return None

        df_concatenado = pd.concat(dataframes, ignore_index=True)
        print("\nArquivos CSV juntados em um DataFrame com sucesso!")
        print(f"Dimensões do DataFrame final concatenado: {df_concatenado.shape[0]} linhas, {df_concatenado.shape[1]} colunas")
        return df_concatenado

    except FileNotFoundError as e:
        print(f"Erro de arquivo não encontrado: {e}. Verifique se todos os arquivos na lista existem.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro ao juntar os arquivos CSV: {e}")
        return None

In [2]:
# --- PARÂMETROS DE EXECUÇÃO ---
# Defina em quantas partes você quer dividir o CSV
N_PARTES = 5

# Caminhos dos arquivos e pastas
arquivo_csv_original = 'ZMays/Agrup/k-mer/k-mer_agrupado.csv'
pasta_saida_partes = 'ZMays/Agrup/k-mer/divididos' # Pasta para salvar as partes

# --- ETAPA 1: Dividindo o CSV em N partes ---
print("--- Etapa 1: Dividindo o CSV ---")
lista_de_partes_csv = dividir_csv_em_n_partes(
    arquivo_original=arquivo_csv_original,
    pasta_saida=pasta_saida_partes,
    n_partes=N_PARTES
)

if lista_de_partes_csv:

    lista_de_zips = compactar_lista_csv(lista_de_partes_csv)
    print(f"Arquivos ZIP criados: {len(lista_de_zips)} de {len(lista_de_partes_csv)}")

    dataframe_final = juntar_n_csvs_em_dataframe(lista_de_partes_csv)

    # --- ETAPA 4: Verificação e Pré-visualização ---
    if dataframe_final is not None:
        print("\n--- Etapa 4: Verificação e Pré-visualização ---")
        print("\nPré-visualização do DataFrame final (primeiras 5 linhas):")
        print(dataframe_final.head())
        print("\nPré-visualização do DataFrame final (últimas 5 linhas):")
        print(dataframe_final.tail())
        print(f"\nDimensões totais verificadas do DataFrame final: {dataframe_final.shape}")

        # Verificação de integridade comparando com o arquivo original
        try:
            
            print("\nVerificando integridade com o arquivo original...")
            df_original_releitura = pd.read_csv(arquivo_csv_original)
            
            if df_original_releitura.equals(dataframe_final):
                print("\nSUCESSO: DataFrame final é idêntico ao DataFrame original!")
                
            else:
                print("\nFALHA: DataFrame final NÃO é idêntico ao DataFrame original. Algo deu errado.")
                
        except Exception as e:
            print(f"\nErro ao tentar verificar com o original: {e}")

--- Etapa 1: Dividindo o CSV ---
Lendo o arquivo original: 'ZMays/Agrup/k-mer/k-mer_agrupado.csv'...
Dimensões do DataFrame original ('k-mer_agrupado.csv'): 98987 linhas, 5462 colunas
------------------------------


c:\Users\Vinícius\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Dividindo em 5 partes na pasta 'ZMays/Agrup/k-mer/divididos'...
Salvando Parte 1: 'k-mer_agrupado_parte_1_de_5.csv' (19798 linhas)
Salvando Parte 2: 'k-mer_agrupado_parte_2_de_5.csv' (19798 linhas)
Salvando Parte 3: 'k-mer_agrupado_parte_3_de_5.csv' (19797 linhas)
Salvando Parte 4: 'k-mer_agrupado_parte_4_de_5.csv' (19797 linhas)
Salvando Parte 5: 'k-mer_agrupado_parte_5_de_5.csv' (19797 linhas)

Arquivo 'ZMays/Agrup/k-mer/k-mer_agrupado.csv' dividido com sucesso!

--- Etapa: Compactando os CSVs divididos ---
Arquivo 'k-mer_agrupado_parte_1_de_5.csv' compactado em 'k-mer_agrupado_parte_1_de_5.zip'
Arquivo 'k-mer_agrupado_parte_2_de_5.csv' compactado em 'k-mer_agrupado_parte_2_de_5.zip'
Arquivo 'k-mer_agrupado_parte_3_de_5.csv' compactado em 'k-mer_agrupado_parte_3_de_5.zip'
Arquivo 'k-mer_agrupado_parte_4_de_5.csv' compactado em 'k-mer_agrupado_parte_4_de_5.zip'
Arquivo 'k-mer_agrupado_parte_5_de_5.csv' compactado em 'k-mer_agrupado_parte_5_de_5.zip'
Arquivos ZIP criados: 5 de 5

--- E

In [3]:
print(dataframe_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98987 entries, 0 to 98986
Columns: 5462 entries, nameseq to label
dtypes: float64(5460), object(2)
memory usage: 4.0+ GB
None


In [4]:
print(dataframe_final)

                     nameseq         A         C         G         T  \
0        5_13437450_13437571  0.327869  0.213115  0.180328  0.278689   
1      2_140681423_140703276  0.274000  0.232360  0.231994  0.261645   
2      4_233416075_233416397  0.328173  0.204334  0.263158  0.204334   
3        3_51304584_51304870  0.365854  0.146341  0.132404  0.355401   
4        9_10968395_10968475  0.209877  0.308642  0.358025  0.123457   
...                      ...       ...       ...       ...       ...   
98982  4_239643403_239643447  0.288889  0.311111  0.222222  0.177778   
98983    3_54350144_54354221  0.315596  0.220942  0.204022  0.259441   
98984    2_20203678_20204129  0.267699  0.232301  0.234513  0.265487   
98985  4_199999392_199999605  0.271028  0.271028  0.257009  0.200935   
98986  6_121108364_121125364  0.282866  0.216223  0.217164  0.283748   

             AA        AC        AG        AT        CA  ...    TTTTCT  \
0      0.107438  0.057851  0.066116  0.099174  0.107438  ... 